In [3]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal


pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line

# from joblib import Memory
# cachedir = '/workspace/cache'
# os.makedirs(cachedir, exist_ok=True)
# memory = Memory(cachedir, verbose=0, bytes_limit=20e9)

%reload_ext autoreload
%autoreload 2


In [5]:
model_name = "tiny-stories-33M"

In [4]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)
model.set_use_attn_result(True)

Loaded pretrained model tiny-stories-33M into HookedTransformer


In [21]:
def load_encoder_cfg(model_name, run_name):
    with open(f"/workspace/{model_name}/{run_name}.json", "r") as f:
        cfg = json.load(f)
    
    return AutoEncoderConfig(
        cfg["layer"], cfg["act"], cfg["expansion_factor"], cfg["l1_coeff"], cfg["d_in"]
    )

data = []
run_data = []
for path in os.listdir(f'/workspace/data/{model_name}'):
    full_path = os.path.join(f'/workspace/data/{model_name}', path)
    run_name = path[:-5]
    cfg = load_encoder_cfg(model_name, run_name)
    with open(full_path, 'r') as f:
        contents = json.load(f)
        data.append(contents)
        run_data.append(f'L{cfg.layer}-{run_name}')

df = pd.DataFrame({'Item': run_data,
                   'Low density cluster': [len(entry['1'])/(len(entry['1']) + len(entry['2']) + len(entry['3']))*100 for entry in data],
                   'Interpretable': [len(entry['2'])/(len(entry['1']) + len(entry['2']) + len(entry['3']))*100 for entry in data],
                   'Uninterpretable': [len(entry['3'])/(len(entry['1']) + len(entry['2']) + len(entry['3']))*100 for entry in data]})
df = df.sort_values('Item')

df_melted = df.melt(id_vars='Item', var_name='Category', value_name='Percentage')

fig = px.bar(df_melted, x='Item', y='Percentage', color='Category', barmode='group', title='Interpretability of dictionary features for TinyStories 33M (N=25)')
fig.show()